# Attempting to load tiled image data using dask

In [1]:
import os
import re
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from natsort import natsorted
import napari
from skimage.io import imread

### Loading image information

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'

In [3]:
%time image_fns = os.listdir(image_dir)

CPU times: user 37.6 ms, sys: 60.2 ms, total: 97.8 ms
Wall time: 95.2 ms


### Characterising the dataset

Number of rows, columns, frames and other params.

In [4]:
fn = image_fns[0]
print(fn)

r03c06f08p02-ch1sk36fk1fl1.tiff


In [5]:
counts = []
for fn in tqdm(image_fns):
    counts.append(list(map(int, re.findall(r'\d+', fn))))

  0%|          | 0/113400 [00:00<?, ?it/s]

In [6]:
df = pd.DataFrame(counts, columns= ['r', 'c', 'f', 'p', 'ch', 'sk', 'fk', 'fl'], dtype=np.uint8)

In [7]:
df

,r,c,f,p,ch,sk,fk,fl
0,3,6,8,2,1,36,1,1
1,6,5,2,1,1,54,1,1
2,5,6,2,1,2,6,1,1
3,4,3,3,3,1,12,1,1
4,4,3,3,1,1,60,1,1
...,...,...,...,...,...,...,...,...
113395,6,5,1,2,2,3,1,1
113396,6,9,4,1,2,74,1,1
113397,3,10,7,2,1,24,1,1
113398,3,7,5,3,1,40,1,1


In [8]:
df.max(axis = 0)

r      6
c     10
f      9
p      3
ch     2
sk    75
fk     1
fl     1
dtype: uint8

In [8]:
df.min(axis = 0)

r     3
c     3
f     1
p     1
ch    1
sk    1
fk    1
fl    1
dtype: uint8

### Focusing on seperating the FOVs (f- note that this is the FOV which are tiled together)

In [50]:
image_fns_fov = {}

for f in range(1,10):
    fns = []
    for fn in image_fns:
        if f'f{f:02d}' in fn:
            fns.append(fn)
    image_fns_fov[f'{f:02d}'] = fns

In [51]:
image_fns_fov

{'01': ['r06c05f01p02-ch1sk44fk1fl1.tiff',
  'r06c07f01p01-ch2sk4fk1fl1.tiff',
  'r03c04f01p03-ch2sk7fk1fl1.tiff',
  'r03c09f01p02-ch1sk14fk1fl1.tiff',
  'r06c06f01p03-ch1sk20fk1fl1.tiff',
  'r05c08f01p02-ch1sk38fk1fl1.tiff',
  'r03c09f01p01-ch1sk68fk1fl1.tiff',
  'r03c03f01p02-ch2sk69fk1fl1.tiff',
  'r05c04f01p02-ch1sk52fk1fl1.tiff',
  'r03c10f01p03-ch1sk33fk1fl1.tiff',
  'r05c04f01p03-ch1sk11fk1fl1.tiff',
  'r06c04f01p02-ch2sk37fk1fl1.tiff',
  'r04c07f01p01-ch1sk12fk1fl1.tiff',
  'r03c09f01p01-ch1sk63fk1fl1.tiff',
  'r04c07f01p03-ch2sk47fk1fl1.tiff',
  'r06c08f01p01-ch1sk4fk1fl1.tiff',
  'r03c10f01p01-ch2sk28fk1fl1.tiff',
  'r05c08f01p02-ch2sk18fk1fl1.tiff',
  'r04c05f01p03-ch1sk68fk1fl1.tiff',
  'r06c07f01p03-ch2sk52fk1fl1.tiff',
  'r03c07f01p02-ch2sk54fk1fl1.tiff',
  'r05c07f01p02-ch2sk38fk1fl1.tiff',
  'r05c04f01p03-ch2sk40fk1fl1.tiff',
  'r04c10f01p03-ch2sk13fk1fl1.tiff',
  'r04c04f01p03-ch2sk55fk1fl1.tiff',
  'r05c05f01p01-ch2sk18fk1fl1.tiff',
  'r04c10f01p02-ch2sk39fk1fl1.tiff'

##### Length of each FOV

In [52]:
for f in image_fns_fov:
    print(len(image_fns_fov[f]))

12600
12600
12600
12600
12600
12600
12600
12600
12600


### Picking one FOV

In [53]:
image_fns_fov['01']

['r06c05f01p02-ch1sk44fk1fl1.tiff',
 'r06c07f01p01-ch2sk4fk1fl1.tiff',
 'r03c04f01p03-ch2sk7fk1fl1.tiff',
 'r03c09f01p02-ch1sk14fk1fl1.tiff',
 'r06c06f01p03-ch1sk20fk1fl1.tiff',
 'r05c08f01p02-ch1sk38fk1fl1.tiff',
 'r03c09f01p01-ch1sk68fk1fl1.tiff',
 'r03c03f01p02-ch2sk69fk1fl1.tiff',
 'r05c04f01p02-ch1sk52fk1fl1.tiff',
 'r03c10f01p03-ch1sk33fk1fl1.tiff',
 'r05c04f01p03-ch1sk11fk1fl1.tiff',
 'r06c04f01p02-ch2sk37fk1fl1.tiff',
 'r04c07f01p01-ch1sk12fk1fl1.tiff',
 'r03c09f01p01-ch1sk63fk1fl1.tiff',
 'r04c07f01p03-ch2sk47fk1fl1.tiff',
 'r06c08f01p01-ch1sk4fk1fl1.tiff',
 'r03c10f01p01-ch2sk28fk1fl1.tiff',
 'r05c08f01p02-ch2sk18fk1fl1.tiff',
 'r04c05f01p03-ch1sk68fk1fl1.tiff',
 'r06c07f01p03-ch2sk52fk1fl1.tiff',
 'r03c07f01p02-ch2sk54fk1fl1.tiff',
 'r05c07f01p02-ch2sk38fk1fl1.tiff',
 'r05c04f01p03-ch2sk40fk1fl1.tiff',
 'r04c10f01p03-ch2sk13fk1fl1.tiff',
 'r04c04f01p03-ch2sk55fk1fl1.tiff',
 'r05c05f01p01-ch2sk18fk1fl1.tiff',
 'r04c10f01p02-ch2sk39fk1fl1.tiff',
 'r04c09f01p03-ch2sk16fk1fl1.ti

In [54]:
gfp_fns = [fn for fn in natsorted(image_fns_fov['01']) if 'ch1' in fn]
rfp_fns = [fn for fn in natsorted(image_fns_fov['01']) if 'ch2' in fn]

In [15]:
gfp_fns

['r03c03f01p01-ch1sk1fk1fl1.tiff',
 'r03c03f01p01-ch1sk2fk1fl1.tiff',
 'r03c03f01p01-ch1sk3fk1fl1.tiff',
 'r03c03f01p01-ch1sk4fk1fl1.tiff',
 'r03c03f01p01-ch1sk5fk1fl1.tiff',
 'r03c03f01p01-ch1sk6fk1fl1.tiff',
 'r03c03f01p01-ch1sk7fk1fl1.tiff',
 'r03c03f01p01-ch1sk8fk1fl1.tiff',
 'r03c03f01p01-ch1sk9fk1fl1.tiff',
 'r03c03f01p01-ch1sk10fk1fl1.tiff',
 'r03c03f01p01-ch1sk11fk1fl1.tiff',
 'r03c03f01p01-ch1sk12fk1fl1.tiff',
 'r03c03f01p01-ch1sk13fk1fl1.tiff',
 'r03c03f01p01-ch1sk14fk1fl1.tiff',
 'r03c03f01p01-ch1sk15fk1fl1.tiff',
 'r03c03f01p01-ch1sk16fk1fl1.tiff',
 'r03c03f01p01-ch1sk17fk1fl1.tiff',
 'r03c03f01p01-ch1sk18fk1fl1.tiff',
 'r03c03f01p01-ch1sk19fk1fl1.tiff',
 'r03c03f01p01-ch1sk20fk1fl1.tiff',
 'r03c03f01p01-ch1sk21fk1fl1.tiff',
 'r03c03f01p01-ch1sk22fk1fl1.tiff',
 'r03c03f01p01-ch1sk23fk1fl1.tiff',
 'r03c03f01p01-ch1sk24fk1fl1.tiff',
 'r03c03f01p01-ch1sk25fk1fl1.tiff',
 'r03c03f01p01-ch1sk26fk1fl1.tiff',
 'r03c03f01p01-ch1sk27fk1fl1.tiff',
 'r03c03f01p01-ch1sk28fk1fl1.tiff',
 

In [105]:
gfp = np.stack([imread(os.path.join(image_dir,fn)) for fn in gfp_fns])

In [18]:
import sys 
from data_utils import convert_size

In [133]:
convert_size(sys.getsizeof(gfp))

'54.75 GB'

In [106]:
viewer = napari.Viewer()

viewer.add_image(gfp)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'gfp' at 0x7faaeb1f9e20>

### Shall I work with one plane or max projection 3 planes?

In [19]:
fns = [os.path.join(image_dir, f'r03c03f01p0{p}-ch1sk1fk1fl1.tiff') for p in range(1,4)]

In [20]:
fns

['/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/r03c03f01p01-ch1sk1fk1fl1.tiff',
 '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/r03c03f01p02-ch1sk1fk1fl1.tiff',
 '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/r03c03f01p03-ch1sk1fk1fl1.tiff']

In [21]:
stack = np.stack([imread(fn) for fn in fns])
max_proj = sum([imread(fn) for fn in fns])

In [128]:
viewer = napari.Viewer()
viewer.add_image(stack)
viewer.add_image(max_proj)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'max_proj' at 0x7f9d1ac5d100>

In [22]:
convert_size(sys.getsizeof(stack))

'26.7 MB'

In [23]:
convert_size(sys.getsizeof(max_proj))

'8.9 MB'

### Working with first plane (p01) for the meantime

In [28]:
%time gfp = np.stack([imread(os.path.join(image_dir, fn)) for fn in gfp_fns if 'p01' in fn])

CPU times: user 2min 23s, sys: 20 s, total: 2min 43s
Wall time: 2min 43s


In [32]:
gfp_p01_fns = [fn for fn in gfp_fns if 'p01' in fn]

In [46]:
viewer = napari.Viewer()
viewer.add_image(gfp)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'gfp' at 0x7f989c0ad970>

# Trying some clever tiling 

In [33]:
import dask

### These are the different wells

In [45]:
print('Min/Max rows')
print(min([int(re.findall(f'r(\d+)', fn)[0]) for fn in gfp_p01_fns]),max([int(re.findall(f'r(\d+)', fn)[0]) for fn in gfp_p01_fns]))
print('Min/Max cols')
print(min([int(re.findall(f'c(\d+)', fn)[0]) for fn in gfp_p01_fns]),max([int(re.findall(f'c(\d+)', fn)[0]) for fn in gfp_p01_fns]))

Min/Max rows
3 6
Min/Max cols
3 10


#### Isolating first frame

In [47]:
gfp_p01_fns

['r03c03f01p01-ch1sk1fk1fl1.tiff',
 'r03c03f01p01-ch1sk2fk1fl1.tiff',
 'r03c03f01p01-ch1sk3fk1fl1.tiff',
 'r03c03f01p01-ch1sk4fk1fl1.tiff',
 'r03c03f01p01-ch1sk5fk1fl1.tiff',
 'r03c03f01p01-ch1sk6fk1fl1.tiff',
 'r03c03f01p01-ch1sk7fk1fl1.tiff',
 'r03c03f01p01-ch1sk8fk1fl1.tiff',
 'r03c03f01p01-ch1sk9fk1fl1.tiff',
 'r03c03f01p01-ch1sk10fk1fl1.tiff',
 'r03c03f01p01-ch1sk11fk1fl1.tiff',
 'r03c03f01p01-ch1sk12fk1fl1.tiff',
 'r03c03f01p01-ch1sk13fk1fl1.tiff',
 'r03c03f01p01-ch1sk14fk1fl1.tiff',
 'r03c03f01p01-ch1sk15fk1fl1.tiff',
 'r03c03f01p01-ch1sk16fk1fl1.tiff',
 'r03c03f01p01-ch1sk17fk1fl1.tiff',
 'r03c03f01p01-ch1sk18fk1fl1.tiff',
 'r03c03f01p01-ch1sk19fk1fl1.tiff',
 'r03c03f01p01-ch1sk20fk1fl1.tiff',
 'r03c03f01p01-ch1sk21fk1fl1.tiff',
 'r03c03f01p01-ch1sk22fk1fl1.tiff',
 'r03c03f01p01-ch1sk23fk1fl1.tiff',
 'r03c03f01p01-ch1sk24fk1fl1.tiff',
 'r03c03f01p01-ch1sk25fk1fl1.tiff',
 'r03c03f01p01-ch1sk26fk1fl1.tiff',
 'r03c03f01p01-ch1sk27fk1fl1.tiff',
 'r03c03f01p01-ch1sk28fk1fl1.tiff',
 

In [44]:
gfp_sk1_fns = [fn for fn in gfp_p01_fns if 'sk1f' in fn]
gfp_sk1_fns

['r03c03f01p01-ch1sk1fk1fl1.tiff',
 'r03c04f01p01-ch1sk1fk1fl1.tiff',
 'r03c05f01p01-ch1sk1fk1fl1.tiff',
 'r03c06f01p01-ch1sk1fk1fl1.tiff',
 'r03c07f01p01-ch1sk1fk1fl1.tiff',
 'r03c08f01p01-ch1sk1fk1fl1.tiff',
 'r03c09f01p01-ch1sk1fk1fl1.tiff',
 'r03c10f01p01-ch1sk1fk1fl1.tiff',
 'r04c03f01p01-ch1sk1fk1fl1.tiff',
 'r04c04f01p01-ch1sk1fk1fl1.tiff',
 'r04c05f01p01-ch1sk1fk1fl1.tiff',
 'r04c06f01p01-ch1sk1fk1fl1.tiff',
 'r04c07f01p01-ch1sk1fk1fl1.tiff',
 'r04c08f01p01-ch1sk1fk1fl1.tiff',
 'r04c09f01p01-ch1sk1fk1fl1.tiff',
 'r04c10f01p01-ch1sk1fk1fl1.tiff',
 'r05c04f01p01-ch1sk1fk1fl1.tiff',
 'r05c05f01p01-ch1sk1fk1fl1.tiff',
 'r05c06f01p01-ch1sk1fk1fl1.tiff',
 'r05c07f01p01-ch1sk1fk1fl1.tiff',
 'r05c08f01p01-ch1sk1fk1fl1.tiff',
 'r05c09f01p01-ch1sk1fk1fl1.tiff',
 'r06c04f01p01-ch1sk1fk1fl1.tiff',
 'r06c05f01p01-ch1sk1fk1fl1.tiff',
 'r06c06f01p01-ch1sk1fk1fl1.tiff',
 'r06c07f01p01-ch1sk1fk1fl1.tiff',
 'r06c08f01p01-ch1sk1fk1fl1.tiff',
 'r06c09f01p01-ch1sk1fk1fl1.tiff']

# Testing different tiling configurations

Working with one well (r03c03) to begin with

In [64]:
#make a list of filenames for all FOV within one well at the first time frame
fns = []
for fov in image_fns_fov:
    for fn in image_fns_fov[fov]:
        if 'r03c03' in fn and 'ch1' in fn:
            fns.append(fn)
fns = natsorted(fns)

In [65]:
fns

['r03c03f01p01-ch1sk1fk1fl1.tiff',
 'r03c03f01p01-ch1sk2fk1fl1.tiff',
 'r03c03f01p01-ch1sk3fk1fl1.tiff',
 'r03c03f01p01-ch1sk4fk1fl1.tiff',
 'r03c03f01p01-ch1sk5fk1fl1.tiff',
 'r03c03f01p01-ch1sk6fk1fl1.tiff',
 'r03c03f01p01-ch1sk7fk1fl1.tiff',
 'r03c03f01p01-ch1sk8fk1fl1.tiff',
 'r03c03f01p01-ch1sk9fk1fl1.tiff',
 'r03c03f01p01-ch1sk10fk1fl1.tiff',
 'r03c03f01p01-ch1sk11fk1fl1.tiff',
 'r03c03f01p01-ch1sk12fk1fl1.tiff',
 'r03c03f01p01-ch1sk13fk1fl1.tiff',
 'r03c03f01p01-ch1sk14fk1fl1.tiff',
 'r03c03f01p01-ch1sk15fk1fl1.tiff',
 'r03c03f01p01-ch1sk16fk1fl1.tiff',
 'r03c03f01p01-ch1sk17fk1fl1.tiff',
 'r03c03f01p01-ch1sk18fk1fl1.tiff',
 'r03c03f01p01-ch1sk19fk1fl1.tiff',
 'r03c03f01p01-ch1sk20fk1fl1.tiff',
 'r03c03f01p01-ch1sk21fk1fl1.tiff',
 'r03c03f01p01-ch1sk22fk1fl1.tiff',
 'r03c03f01p01-ch1sk23fk1fl1.tiff',
 'r03c03f01p01-ch1sk24fk1fl1.tiff',
 'r03c03f01p01-ch1sk25fk1fl1.tiff',
 'r03c03f01p01-ch1sk26fk1fl1.tiff',
 'r03c03f01p01-ch1sk27fk1fl1.tiff',
 'r03c03f01p01-ch1sk28fk1fl1.tiff',
 

###  Merging p1-3, loading as a stack and viewing in napari grid mode

In [84]:
fov_stacks, plane_stack = {}, {}
for f in range(1,10):
    for p in tqdm(range(1,4)):
        plane_stack[p] = np.stack([imread(os.path.join(image_dir, fn)) for fn in fns if f'p0{p}' in fn and f'f{f:02d}' in fn])
    fov_stacks[f] = sum(plane_stack.values())

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [85]:
fov_stacks

{1: array([[[ 326,  328,  319, ...,  510,  539,  548],
         [ 323,  333,  313, ...,  476,  505,  521],
         [ 322,  326,  324, ...,  465,  478,  492],
         ...,
         [2007, 2135, 2335, ...,  323,  311,  321],
         [2050, 2255, 2458, ...,  320,  286,  293],
         [2153, 2336, 2593, ...,  318,  320,  314]],
 
        [[ 312,  322,  329, ...,  333,  326,  340],
         [ 325,  330,  323, ...,  304,  316,  315],
         [ 309,  322,  330, ...,  302,  313,  305],
         ...,
         [ 775,  742,  672, ...,  713,  671,  697],
         [ 834,  818,  747, ...,  743,  704,  712],
         [ 926,  894,  838, ...,  756,  759,  773]],
 
        [[ 318,  318,  306, ...,  337,  319,  336],
         [ 330,  327,  307, ...,  323,  319,  326],
         [ 323,  321,  327, ...,  316,  322,  324],
         ...,
         [ 587,  574,  580, ...,  412,  425,  432],
         [ 615,  621,  641, ...,  396,  394,  381],
         [ 630,  625,  627, ...,  377,  408,  422]],
 
        ..

In [86]:
viewer = napari.Viewer()
viewer.add_image(np.stack(fov_stacks[1]))

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'Image' at 0x7f993ebaafa0>

In [94]:
viewer = napari.Viewer()
viewer.add_image(np.concatenate((fov_stacks[1], fov_stacks[2]), axis = 2))

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'Image' at 0x7f9914903460>

### Loading one image each next to one another with filename overlaid

In [107]:
fns

['r03c03f01p01-ch1sk1fk1fl1.tiff',
 'r03c03f01p01-ch1sk2fk1fl1.tiff',
 'r03c03f01p01-ch1sk3fk1fl1.tiff',
 'r03c03f01p01-ch1sk4fk1fl1.tiff',
 'r03c03f01p01-ch1sk5fk1fl1.tiff',
 'r03c03f01p01-ch1sk6fk1fl1.tiff',
 'r03c03f01p01-ch1sk7fk1fl1.tiff',
 'r03c03f01p01-ch1sk8fk1fl1.tiff',
 'r03c03f01p01-ch1sk9fk1fl1.tiff',
 'r03c03f01p01-ch1sk10fk1fl1.tiff',
 'r03c03f01p01-ch1sk11fk1fl1.tiff',
 'r03c03f01p01-ch1sk12fk1fl1.tiff',
 'r03c03f01p01-ch1sk13fk1fl1.tiff',
 'r03c03f01p01-ch1sk14fk1fl1.tiff',
 'r03c03f01p01-ch1sk15fk1fl1.tiff',
 'r03c03f01p01-ch1sk16fk1fl1.tiff',
 'r03c03f01p01-ch1sk17fk1fl1.tiff',
 'r03c03f01p01-ch1sk18fk1fl1.tiff',
 'r03c03f01p01-ch1sk19fk1fl1.tiff',
 'r03c03f01p01-ch1sk20fk1fl1.tiff',
 'r03c03f01p01-ch1sk21fk1fl1.tiff',
 'r03c03f01p01-ch1sk22fk1fl1.tiff',
 'r03c03f01p01-ch1sk23fk1fl1.tiff',
 'r03c03f01p01-ch1sk24fk1fl1.tiff',
 'r03c03f01p01-ch1sk25fk1fl1.tiff',
 'r03c03f01p01-ch1sk26fk1fl1.tiff',
 'r03c03f01p01-ch1sk27fk1fl1.tiff',
 'r03c03f01p01-ch1sk28fk1fl1.tiff',
 

In [109]:
tile1 = imread(os.path.join(image_dir, 'r03c03f01p01-ch1sk1fk1fl1.tiff'))

In [108]:
tile2 = np.concatenate((np.zeros(fov_stacks[2][0].shape),imread(os.path.join(image_dir, 'r03c03f02p01-ch1sk1fk1fl1.tiff'))), axis = 1)

In [111]:
viewer = napari.Viewer()

viewer.add_image(tile1, name = 'tile 1', blending = 'additive')
viewer.add_image(tile2, name = 'tile 2', blending = 'additive')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'tile 2' at 0x7f9915f08580>

### Loading time series 

In [112]:
tile1 = np.stack([imread(os.path.join(image_dir, f'r03c03f01p01-ch1sk{t}fk1fl1.tiff')) for t in range(1,76)])

In [113]:
tile2 = np.stack([np.concatenate((np.zeros(fov_stacks[2][0].shape),imread(os.path.join(image_dir, f'r03c03f02p01-ch1sk{t}fk1fl1.tiff'))), axis = 1) for t in range(1,76)])

In [114]:
viewer = napari.Viewer()

viewer.add_image(tile1, name = 'tile 1', blending = 'additive')
viewer.add_image(tile2, name = 'tile 2', blending = 'additive')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'tile 2' at 0x7f992f6751f0>

### Trying different tiles 

In [116]:
tile_number = f = 2
axis = 1 ## 1 is horizontal, 2 is vertical

In [117]:
tile1 = np.stack([imread(os.path.join(image_dir, f'r03c03f0{f}p01-ch1sk{t}fk1fl1.tiff')) for t in range(1,76)])

In [ ]:
overlap_shape = 

In [118]:
%time tile2 = np.stack([np.concatenate((np.zeros(fov_stacks[2][0].shape),imread(os.path.join(image_dir, f'r03c03f0{f+1}p01-ch1sk{t}fk1fl1.tiff'))), axis = axis) for t in range(1,76)])

CPU times: user 6.99 s, sys: 1.8 s, total: 8.79 s
Wall time: 8.78 s


In [121]:
viewer = napari.Viewer()

viewer.add_image(tile1, name = 'tile 1', blending = 'additive',)
viewer.add_image(tile2, name = 'tile 2', blending = 'additive', )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'tile 2' at 0x7f98e3451b20>